In [1]:
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, GLPK
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, r2_score

References

* https://apmonitor.com/che263/index.php/Main/PythonOptimization (initial formulation)
* https://stackoverflow.com/questions/43702352/maximize-optimization-using-scipy (-1 trick)
* https://het.as.utexas.edu/HET/Software/Scipy/generated/scipy.optimize.minimize.html ((0,None) trick)
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize API Ref
* https://stackoverflow.com/questions/17009774/quadratic-program-qp-solver-that-only-depends-on-numpy-scipy Plot
* https://realpython.com/python-scipy-cluster-optimize/ Walkthrough 

# 1.  
An engineer at Fertilizer Company has synthesized a sensational new fertilizer made of just two interchangeable 
basic raw materials.  The company wants to take advantage of this opportunity and produce as much as possible of 
the new fertilizer.  The company currently has \\$40,000 to buy raw materials at a unit price of \\$8000 and \\$5000 per unit, respectively.  When amounts x1 and x2 of the basic raw materials are combined, a quantity q of fertilizer results given by:  𝑞=4𝑥1 +2𝑥2 −0.5𝑥12 −0.25𝑥22 
 
## Part A:    
Formulate as a constrained nonlinear program.  Clearly indicate the variables, objective function, and 
constraints.   
 
## Part B:    
Solve the Program (provide exact values for all variables and the optimal objective function). 

In [2]:
def objective(x):
    '''
    When amounts x1 and x2 of the basic raw materials are combined, a quantity q of fertilizer results
    If this were a min problem, it would simply be q = 4*x1 + 2*x2 - 0.5*x1**2 - 0.25*x2**2
    Since this is a max, we multiply it by -1
    '''
    x1 = x[0]
    x2 = x[1]
    q = -1 * (4*x1 + 2*x2 - 0.5*x1**2 - 0.25*x2**2)
    return q

def constraint1(x):
    '''
    The company currently has $40,000 to buy raw materials at a unit price of $8000 and $5000 per unit, respectively
    '''
    x1 = x[0]
    x2 = x[1]
    return 8000*x1 + 5000*x2 - 40000


# initial guesses
x0 = [0, 0]

# show initial objective
print('Initial Objective: ' + str(objective(x0)))

# optimize
b = (0.0,None)
bnds = (b, b) ## greater than 0 constraints
con1 = {'type': 'ineq', 'fun': constraint1}
cons = ([con1])
solution = minimize(objective,x0,method='SLSQP', bounds=bnds,constraints=cons)
x = solution.x

# show final objective
print('Final Objective: ' + str(objective(x) * -1))

# print solution
print('Solution')
print('x1 = ' + str(x[0]))
print('x2 = ' + str(x[1]))

Initial Objective: -0.0
Final Objective: 11.999999998793982
Solution
x1 = 4.000013055847088
x2 = 4.000066956398335


# 2.  

The area of a triangle with sides of length a, b, and c is √𝑠(𝑠−𝑎)(𝑠−𝑏)(𝑠−𝑐), where s is half the perimeter 
of the triangle.  We have 60 feet of fence and want to fence a triangular-shaped area.   

## Part A:     
Formulate the problem as a constrained nonlinear program that will enable us to maximize the area of the 
fenced area, with constraints.  Clearly indicate the variables, objective function, and constraints. 
 
Hint: The length of a side of a triangle must be less than or equal to the sum of the lengths of the other two sides. 
 
## Part B: 
Solve the Program (provide exact values for all variables and the optimal objective function). 


In [4]:
def triangle_area(x, *args):
    '''
    a, b, and c are the side of a triangle
    s is half the perimeter of the triangle
    '''
    a, b, c = x
    s = (a + b + c) / 2
    area = np.sqrt(s * (s - a) * (s - b) * (s - c))
    return -area

def triangle_constraint(x):
    a, b, c = x
    return [60 - (a + b + c), a + b - c, a + c - b, b + c - a, a, b, c]

x0 = [10, 10, 10] # initial guess for the lengths of the sides

# show initial objective
print('Initial Objective: ' + str(objective(x0)))

# optimize
b = (0.0, 60)
bnds = (b, b, b) ## greater than 0 constraints
con1 = {'type': 'ineq', 'fun': triangle_constraint}
cons = ([con1])
solution = minimize(triangle_area, x0, method='SLSQP', bounds=bnds, constraints=cons)
a, b, c = solution.x

# show final objective
print('Final Objective (area): ' + str(triangle_area(solution.x) * -1))

# print solution
print('Solution')
print('x1 = ' + str(a))
print('x2 = ' + str(b))
print('x3 = ' + str(c))

Initial Objective: 15.0
Final Objective (area): 173.20508075688707
Solution
x1 = 19.999999999999964
x2 = 19.99999999999996
x3 = 19.99999999999996


# 3
The Tiny Toy Company makes three types of new toys: the tiny tank, the tiny truck, and the tiny turtle. Plastic 
used in one unit of each is 1.5, 2.0 and 1.0 pounds, respectively. Rubber for one unit of each toy is 0.5, 0.5, and 1.0 pounds,  respectively.  Also,  each  tank  uses  0.3  pounds  of  metal  and  the  truck  uses  0.6  pounds  of  metal  during production. The average weekly availability for plastic is 16,000 pounds, 9,000 pounds of metal, and 5,000 pounds of  rubber.  It  takes  two  hours  of  labor  to  make  one  tank,  two  hours  for  one  truck,  and  one  hour  for  a  turtle.  

The company allows no more than 40 hours a week for production (priority \#1). Finally, the cost of manufacturing one 
tank is \\$7, 1 truck is \\$5 and 1 turtle is \\$4; a target budget of \\$164,000 is initially used as a guideline for the company 
to follow. 
 

 


a) Minimize over-utilization of the weekly available supply of materials used in making the toys and place 
twice as much emphasis on the plastic (priority \#2)

b) Minimize the under and over-utilization of the budget. Maximize available labor hour usage (priority \#3).
 
Formulate the above decision problem as a single linear goal program. Clearly identify your achievement vector (i.e., hierarchy of priority levels for the goals).  Do not solve



#### Variables:
    x1 = tank
    x2 = truck
    x3 = turtle

#### Constraints  
  7x1 + 5x2 + 4x3 <= 164,000 (budget objective)    
  2x1 + 2x2 + x3 <= 40 (hours of labor constraint)    
  1.5x1 + 2x2 + x3 <= 16,000 (plastic constraint)    
  0.5x1 + 0.5x2 + x3 <= 5,000 (rubber constraint)    
  0.3x1 + 0.6x2 <= 9,000 (metal constraint)    
  x1 >= 0, x2 >= 0, x3 >= 0 (non-negativity constraints)    
  
  Goal 1: Taken care of by constraints (The company allows no more than 40 hours a week for production)    
  Goal 2: Taken care of by constraints (minimize overutilization of supply and place twice as much emphasis on the plastic)    
  Goal 3: Taken care of by constraints (Maximize available labor hour usage)    
  Goal 4: Taken care of by constraints (Minimize the over-utilization of the budget)    
  Goal 5: Taken care of by constraints (Minimize the under-utilization of the budget)    

#### Goal Programming Form
7x1 + 5x2 + 4x3 + η1 – ρ1 = 164000     
1.5x1 + 2x2 + x3 + η2 – ρ2 = 16000     
0.5x1 + 0.5x2 + x3 + η3 – ρ3 = 5000     
0.3x1 + 0.6x2 + η4 – ρ4 = 9000     
2x1 + 2x2 + x3 + η5 – ρ5 = 40         
x1, x2, x3 ≥ 0 and ηi, ρi ≥ 0 for all i 

#### Achievement Vector
$$\text{Lex Min}\begin{pmatrix}
ρ5 \\ 2ρ2 + ρ3 + ρ4 \\ η5 \\ η1 \\ ρ1
\end{pmatrix}$$


# 4.  

XYZ Company is planning an advertising campaign for its new product. The media considered are television and 
radio.  Rated  exposures  per  thousand  dollars  of  advertising  expenditure  are  10,000  for  TV  and  7,500  for  radio. 

Management has agreed that the  campaign cannot be judged successful if total exposures are  under 750,000. The 
campaign  would  be  viewed  as  superbly  successful  if  1  million  exposures  occurred.  In  addition,  the  company  has realized that the two most important audiences for its product are persons 18 to 21 years of age and persons 25 to 30 years of age. The following table estimates the number of individuals in the two age groups expected to be exposed to advertisements per \\$ 1,000 of expenditures: 

|Age	|Television	|Radio|
|-------------------|-----------|-----------|
|18-21	|2,500	|3,000|
|25-30	|3,000	|1,500|

Management has rank ordered five goals it wishes to achieve, arranged from highest to lowest priorities.   
1. Achieve total exposures of at least 750,000 persons. 
1. Avoid expenditures of more than \\$100,000. 
1. Avoid expenditures of more than \\$70,000 for television advertisements. 
1. Achieve at least 1 million total exposures. 
1. Reach at least 250,000 persons in each of the two age groups, 18-21 and 25-30 years. In addition, 
management realizes and wishes to account for the fact that the purchasing power of the 25-30 age group is 
twice that of the 18-21 age group. 
 
Formulate the above decision problem as a single linear goal program. Clearly identify your achievement vector (i.e., 
hierarchy of priority levels for the goals).  Do not solve. 

### Variables:    
x1 = amount of money to spend on TV ads, in thousands
x2 = amount of money to spend on radio ads, in thousands

### Constraints
__Goal 1 - Minimum Total Exposures:__     
10,000x1 + 7,500x2 >= 750,000    

__Goal 2 - Total Expenditures:__     
x1 + x2 <= 100,000         

__Goal 3 - TV Expenditures:__           
x1 <= 70,000        

__Goal 4 - Superb Total Exposures:__     
10,000x1 + 7,500x2 >= 1,000,000     

__Goal 5 and 6 - Age Group Exposures:__     
18-21 years: 2,500x1 + 3,000x2 >= 250,000      
25-30 years: 3,000x1 + 1,500x2 >= 250,000    

__Non-negativity Constraints:__
x1 >= 0, x2 >= 0 (non-negativity constraints)


### Goal Programming Form
__Priority 1 - Minimum Total Exposures:__       
10,000x1 + 7,500x2 + η1 – ρ1 = 750,000    
__Priority 2 - Total Expenditures:__       
x1 + x2 + η2 – ρ2 = 100,000    
__Priority 3 - TV Expenditures:__       
x1 + η3 – ρ3 = 70,000  
__Priority 4 - Superb Total Exposures:__       
10,000x1 + 7,500x2 + η4 – ρ4 = 1,000,000   
__Priority 5 - Age Group Exposures:__    
25-30 years: 3,000x1 + 1,500x2 + η5 – ρ5 = 250,000  
18-21 years: 2,500x1 + 3,000x2 + η6 – ρ6 = 250,000  

x1, x2 ≥ 0 and ηi, ρi ≥ 0 for all i 

### Achievement Vector    
$$\text{Lex Min}\begin{pmatrix}
η1 \\ ρ2 \\ ρ3 \\ η4 \\ 2η5 + η6
\end{pmatrix}$$

# 5

A large food chain owns a number of pharmacies that operate in a variety of settings. Some are situated in small 
towns and are open for only 8 hours a day, 5 days per week. Others are located in shopping malls and are open for 
longer hours. The analysts on the corporate staff would like to develop a model to show how a store’s revenues 
depend on the number of hours that it is open. They have collected the following information from a sample of stores. 

|Hours of Operation |Average Revenue (\\$) |
|-------------------|-----------|
|40 |5958 |
|44 |6662 |
|48 |6004 |
|48 |6011 |
|60 |7250 |
|70 |8632 |
|72 |6964 | 
|90 |11097| 
|100 |9107| 
|168 |11498 |

__a)__ Use a linear function (e.g., y = ax + b; where a and b are parameters to optimize) to represent the relationship 
between revenue and operating hours and find the values of the parameters using the nonlinear solver that 
provide the best fit to the given data. What revenue does your model predict for 120 hours?     

__b)__ Suggest a two-parameter nonlinear model (e.g., y = axb; where a and b are parameters to optimize) for the 
same relationship and find the parameters using the Nonlinear Solver that provide the best fit. What revenue 
does your model predict for 120 hours? Which if the models in (a) and (b) do you prefer and why?      
 
Your solutions for (a) and (b) should contain a detailed spreadsheet model (where the decision variables, parameters, 
objective function and constraints are identified and explained), as well as answers to the questions posed. You may 
use Microsoft Excel, Python, or R to solve. 

In [5]:
## Linear
### a) Use a linear function (e.g., y = ax + b; where a and b are parameters to optimize) to represent the 
### relationship between revenue and operating hours and find the values of the parameters using the 
## nonlinear solver that provide the best fit to the given data. What revenue does your model predict for 120 hours?

# Sample data
hours = np.array([40, 44, 48, 48, 60, 70, 72, 90, 100, 168])
revenue = np.array([5958, 6662, 6004, 6011, 7250, 8632, 6964, 11097, 9107, 11498])

# Objective 
def lin_objective(params):
    '''linear objective'''
    a, b = params
    y_pred = a * hours + b
    return np.sum((revenue - y_pred)**2)

# Initial guess 
x0 = [10, 10]

# optimize
b = (0.0, None)
bnds = (b, b) ## greater than 0 constraints
solution = minimize(lin_objective, x0, method='SLSQP', bounds=bnds)

# Extract the optimized parameters
a_solution, b_solution = solution.x

# Predict revenue for 120 hours of operation
hrs_to_predict = 120
revenue_pred = a_solution * hrs_to_predict + b_solution

print(f"Predicted Revenue for {hrs_to_predict} Hours of Operation:", revenue_pred)


Predicted Revenue for 120 Hours of Operation: 10083.538614114037


In [6]:
## Non linear
#### b) Suggest a two-parameter nonlinear model (e.g., y = axb; where a and b are parameters to optimize) 
### for the same relationship and find the parameters using the Nonlinear Solver that provide the best fit. 
### What revenue does your model predict for 120 hours? 

# Objective 
def non_lin_objective(params):
    a, b = params
    y_pred = a * hours**b
    return np.sum((revenue - y_pred)**2)

# Initial guess 
x0 = [1, 1]

# optimize
solution_non_lin = minimize(non_lin_objective, x0)

# Extract the optimized parameters
a_solution_non_lin, b_solution_non_lin = solution_non_lin.x

# Predict revenue for 120 hours of operation
hrs_to_predict = 120
revenue_pred_non_lin = a_solution_non_lin * hrs_to_predict** b_solution_non_lin

print(f"Predicted Revenue for {hrs_to_predict} Hours of Operation:", revenue_pred_non_lin)


Predicted Revenue for 120 Hours of Operation: 10260.874030232746


__Which of the models in (a) and (b) do you prefer and why?__

I would normally prefer the linear model: it is simpler to understand.  However, if I calculate r squared and RSME for each of these models by predicting the actuals from the dataset, the non-linear model performs slightly better.  See below.  Therefore, in this case, I prefer the non-linear model.

In [24]:
non_linear_predictions = [a_solution_non_lin * i** b_solution_non_lin for i in hours]
linear_predictions = [a_solution * i + b_solution for i in hours]

# Calculate the R^2 score
r2_non_lin = r2_score(revenue, non_linear_predictions)
r2_lin = r2_score(revenue, linear_predictions)

# Calculate the root mean squared error
rmse_non_lin = math.sqrt(mean_squared_error(revenue, non_linear_predictions))
rmse_lin = np.sqrt(mean_squared_error(revenue, linear_predictions))

print("Predicted revenue for 120 hours for linear model:", a_solution * hrs_to_predict + b_solution)
print("Predicted revenue for 120 hours for non-linear model:", a_solution_non_lin * hrs_to_predict**b_solution_non_lin)
print(40*"*")
print("R^2 score for linear model:", r2_lin)
print("R^2 score for non-linear model:", r2_non_lin)
print(40*"*")
print("Root mean squared error for linear model:", rmse_lin)
print("Root mean squared error for non-linear model:", rmse_non_lin)


Predicted revenue for 120 hours for linear model: 10083.538614114037
Predicted revenue for 120 hours for non-linear model: 10260.874030232746
****************************************
R^2 score for linear model: 0.7623485024350101
R^2 score for non-linear model: 0.8059002472714212
****************************************
Root mean squared error for linear model: 961.7560791545825
Root mean squared error for non-linear model: 869.1748543901159
